# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [18]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [19]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(5) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.8325893139136817),
 (1, 1.8325893139136817),
 (2, 1.8325893139136817),
 (3, 1.8325893139136817),
 (4, 1.8325893139136817)]

## Inverted index

In [20]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [21]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [22]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [23]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('it', 18)]),
 (1, [('is', 18), ('what', 10)])]

## TeraSort

In [29]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.014057255910709499),
   (None, 0.04556278859736984),
   (None, 0.06885362893462688),
   (None, 0.0783381837226611),
   (None, 0.09576205279058592),
   (None, 0.14838271640603473),
   (None, 0.1547131177207256),
   (None, 0.16206753854237388),
   (None, 0.16603490430665968),
   (None, 0.21955168254073554),
   (None, 0.2719883665586824),
   (None, 0.36083707179332836),
   (None, 0.4008996224864819),
   (None, 0.43229702191861397)]),
 (1,
  [(None, 0.5272478752686875),
   (None, 0.5448950957973508),
   (None, 0.5545710678322974),
   (None, 0.5832762931075615),
   (None, 0.6775013865124019),
   (None, 0.774449285748408),
   (None, 0.8084251186328072),
   (None, 0.8571055989289278),
   (None, 0.8616559467673063),
   (None, 0.8745151150738617),
   (None, 0.9197545303439842),
   (None, 0.9209631886103619),
   (None, 0.9275102280624786),
   (None, 0.9421943916367262),
   (None, 0.9642913324471696),
   (None, 0.9753501319154418)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [30]:
import random  # Импортируем модуль random для генерации случайных чисел

# Создаем список из 100 случайных чисел в диапазоне от 0 до 100
input_list = [random.randint(0, 100) for _ in range(100)]

def RECORDREADER():  # Функция для чтения данных
  for value in input_list:  # Проходим по каждому значению в списке
    yield (0, value)  # Возвращаем пару (0, значение)

def MAP(id:int, value:int):  # Функция map, которая просто возвращает полученные данные
  yield (id, value)  # Возвращаем пару (id, значение)

def REDUCE(id:int, values:list):  # Функция reduce, которая находит максимальное значение
  yield (id, max(values))  # Возвращаем пару (id, максимальное значение)

# Применяем функции RECORDREADER, MAP и REDUCE к нашим данным
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)  # Преобразуем результат в список
output  # Выводим результат

[(0, 100)]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [31]:
# Создаем список из чисел
input_list = [1, 3, 4, 5, 6]

def RECORDREADER():  # Функция для чтения данных
  for value in input_list:  # Проходим по каждому значению в списке
    yield (0, value)  # Возвращаем пару (0, значение)

def MAP(idx:int, value:int):  # Функция map, которая просто возвращает полученные данные
   yield (idx, value)  # Возвращаем пару (idx, значение)

def REDUCE(idx:int, values:list):  # Функция reduce, которая вычисляет среднее значение
  yield (idx, sum(values)/len(values))  # Возвращаем пару (idx, среднее значение)

# Применяем функции RECORDREADER, MAP и REDUCE к нашим данным
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)  # Преобразуем результат в список
output  # Выводим результат

[(0, 3.8)]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [32]:
# Создаем список пользователей
data_list = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def RECORDREADER(key:str):  # Функция для чтения данных
  for user in data_list:  # Проходим по каждому пользователю в списке
    yield (getattr(user, key), user)  # Возвращаем пару (значение атрибута, пользователь)

# Функция для группировки данных по ключу
def groupByKey(iterable):
  groups = {}  # Создаем словарь для групп
  for key, value in sorted(iterable, key=lambda x:x[0]):  # Сортируем данные по ключу
    if key not in groups.keys():  # Если ключа еще нет в словаре
      groups[key] = []  # Создаем новую группу
    groups[key].append(value)  # Добавляем значение в группу
  return groups.items()  # Возвращаем группы

# Применяем функции RECORDREADER и MAP к нашим данным и группируем результаты по id
print(list(groupByKey(flatten(map(lambda x: MAP(*x), RECORDREADER("id"))))), end="\n\n")
# Применяем функции RECORDREADER и MAP к нашим данным и группируем результаты по полу
print(list(groupByKey(flatten(map(lambda x: MAP(*x), RECORDREADER("gender"))))), end="\n\n")
# Применяем функции RECORDREADER и MAP к нашим данным и группируем результаты по возрасту
print(list(groupByKey(flatten(map(lambda x: MAP(*x), RECORDREADER("age"))))))

[(0, [User(id=0, age=55, social_contacts=20, gender='male')]), (1, [User(id=1, age=25, social_contacts=240, gender='female')]), (2, [User(id=2, age=25, social_contacts=500, gender='female')]), (3, [User(id=3, age=33, social_contacts=800, gender='female')])]

[('female', [User(id=1, age=25, social_contacts=240, gender='female'), User(id=2, age=25, social_contacts=500, gender='female'), User(id=3, age=33, social_contacts=800, gender='female')]), ('male', [User(id=0, age=55, social_contacts=20, gender='male')])]

[(25, [User(id=1, age=25, social_contacts=240, gender='female'), User(id=2, age=25, social_contacts=500, gender='female')]), (33, [User(id=3, age=33, social_contacts=800, gender='female')]), (55, [User(id=0, age=55, social_contacts=20, gender='male')])]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [33]:
import numpy as np  # Импортируем модуль numpy для математических операций

# Создаем список пользователей
data_list = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=3, age=33, gender='female', social_contacts=800)
]

maps = 2  # Количество map-задач
reducers = 2  # Количество reduce-задач

# Функция для разделения данных на части
def INPUTFORMAT():
  global maps  # Используем глобальную переменную maps

  # Функция для чтения данных
  def RECORDREADER(split):
    for user in split:  # Проходим по каждому пользователю в списке
      yield (user.id, user)  # Возвращаем пару (id пользователя, пользователь)

  # Размер каждого раздела данных
  split_size = int(np.ceil(len(data_list)/maps))
  # Разделяем данные на части и возвращаем итератор по каждой части
  for i in range(0, len(data_list), split_size):
    yield RECORDREADER(data_list[i:i+split_size])

# Функция map, которая просто возвращает полученные данные
def MAP(userId:int, user:User):
  yield (userId, user)  # Возвращаем пару (id пользователя, пользователь)

# Функция reduce, которая удаляет дубликаты пользователей
def REDUCE(_, usersList:list):
  yield (list(set(usersList)))  # Возвращаем список уникальных пользователей

# Применяем функции INPUTFORMAT, MAP и REDUCE к нашим данным
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
# Преобразуем результат в список пар (id раздела, раздел)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output  # Выводим результат

4 key-value pairs were sent over a network.


[(0, [[User(id=0, age=55, social_contacts=20, gender='male')]]),
 (1,
  [[User(id=1, age=25, social_contacts=240, gender='female')],
   [User(id=3, age=33, social_contacts=800, gender='female')]])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [34]:
# Создаем список пользователей
data_list = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=240),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def RECORDREADER():  # Функция для чтения данных
  for user in data_list:  # Проходим по каждому пользователю в списке
    yield (user.id, user)  # Возвращаем пару (id пользователя, пользователь)

# Функция map, которая возвращает пользователей младше 45 лет
def MAP(userId:int, user:User):
  if user.age < 45:  # Если возраст пользователя меньше 45 лет
    yield (user, user)  # Возвращаем пару (пользователь, пользователь)

# Функция reduce, которая просто возвращает полученного пользователя
def REDUCE(userId:int, user:User):
  yield user  # Возвращаем пользователя

# Применяем функции RECORDREADER, MAP и REDUCE к нашим данным
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)  # Преобразуем результат в список
output  # Выводим результат

[[User(id=1, age=25, social_contacts=240, gender='female')],
 [User(id=2, age=25, social_contacts=240, gender='female')],
 [User(id=3, age=33, social_contacts=800, gender='female')]]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [35]:
# Определяем класс UserProjection с полями id и gender
class UserProjection(NamedTuple):
  id: int
  gender: str

# Создаем список пользователей
data_list = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=240),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def RECORDREADER():  # Функция для чтения данных
  for user in data_list:  # Проходим по каждому пользователю в списке
    yield (user.id, user)  # Возвращаем пару (id пользователя, пользователь)

# Функция map, которая преобразует пользователя в объект UserProjection
def MAP(userId:int, user:User):
  new_user = UserProjection(userId, user.gender)  # Создаем нового пользователя с полями id и gender
  yield(new_user, new_user)  # Возвращаем пару (новый пользователь, новый пользователь)

# Функция reduce, которая просто возвращает полученного пользователя
def REDUCE(key:UserProjection, usersList:list):
  yield (key, key)  # Возвращаем пару (ключ, ключ)

# Применяем функции RECORDREADER, MAP и REDUCE к нашим данным
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)  # Преобразуем результат в список
output  # Выводим результат

[(UserProjection(id=0, gender='male'), UserProjection(id=0, gender='male')),
 (UserProjection(id=1, gender='female'),
  UserProjection(id=1, gender='female')),
 (UserProjection(id=2, gender='female'),
  UserProjection(id=2, gender='female')),
 (UserProjection(id=3, gender='female'),
  UserProjection(id=3, gender='female'))]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [36]:
# Создаем два списка пользователей
data_list1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=240),
    User(id=3, age=33, gender='female', social_contacts=800)
]

data_list2 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=4, age=25, gender='female', social_contacts=240),
    User(id=5, age=25, gender='female', social_contacts=240),
    User(id=6, age=33, gender='female', social_contacts=800)
]

# Объединяем два списка пользователей в один
data_list = [data_list1, data_list2]

def RECORDREADER():  # Функция для чтения данных
  for list_ in data_list:  # Проходим по каждому списку в общем списке
    for user in list_:  # Проходим по каждому пользователю в списке
      yield (user.id, user)  # Возвращаем пару (id пользователя, пользователь)

# Функция map, которая просто возвращает полученные данные
def MAP(userId:int, user:User):
  yield(user, user)  # Возвращаем пару (пользователь, пользователь)

# Функция reduce, которая просто возвращает полученного пользователя
def REDUCE(key:User, usersList:list):
  yield (key, key)  # Возвращаем пару (ключ, ключ)

# Применяем функции RECORDREADER, MAP и REDUCE к нашим данным
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)  # Преобразуем результат в список
output  # Выводим результат

[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male')),
 (User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female')),
 (User(id=2, age=25, social_contacts=240, gender='female'),
  User(id=2, age=25, social_contacts=240, gender='female')),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female')),
 (User(id=4, age=25, social_contacts=240, gender='female'),
  User(id=4, age=25, social_contacts=240, gender='female')),
 (User(id=5, age=25, social_contacts=240, gender='female'),
  User(id=5, age=25, social_contacts=240, gender='female')),
 (User(id=6, age=33, social_contacts=800, gender='female'),
  User(id=6, age=33, social_contacts=800, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [37]:
# Создаем два списка пользователей
data_list1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=240),
    User(id=3, age=33, gender='female', social_contacts=800)
]

data_list2 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=4, age=25, gender='female', social_contacts=240),
    User(id=5, age=25, gender='female', social_contacts=240),
    User(id=6, age=33, gender='female', social_contacts=800)
]

# Объединяем два списка пользователей в один
data_list = [data_list1, data_list2]

def RECORDREADER():  # Функция для чтения данных
  for list_ in data_list:  # Проходим по каждому списку в общем списке
    for user in list_:  # Проходим по каждому пользователю в списке
      yield (user.id, user)  # Возвращаем пару (id пользователя, пользователь)

# Функция map, которая просто возвращает полученные данные
def MAP(userId:int, user:User):
  yield(user, user)  # Возвращаем пару (пользователь, пользователь)

# Функция reduce, которая возвращает пользователя, если он встречается в списке дважды
def REDUCE(key:User, usersList:list):
  if len(usersList) == 2:  # Если пользователь встречается в списке дважды
    yield (key, key)  # Возвращаем пару (ключ, ключ)

# Применяем функции RECORDREADER, MAP и REDUCE к нашим данным
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)  # Преобразуем результат в список
output  # Выводим результат

[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male'))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [38]:
# Создаем два списка пользователей
data_list1 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=240),
    User(id=3, age=33, gender='female', social_contacts=800)
]

data_list2 = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=4, age=25, gender='female', social_contacts=240),
    User(id=5, age=25, gender='female', social_contacts=240),
    User(id=6, age=33, gender='female', social_contacts=800)
]

# Объединяем два списка пользователей в один
data_list = [data_list1, data_list2]

def RECORDREADER():  # Функция для чтения данных
  for idx, list_ in enumerate(data_list):  # Проходим по каждому списку в общем списке
    for user in list_:  # Проходим по каждому пользователю в списке
      yield (idx, user)  # Возвращаем пару (индекс списка, пользователь)

# Функция map, которая возвращает полученные данные
def MAP(setId:int, user:User):
  yield(user, setId)  # Возвращаем пару (пользователь, индекс списка)

# Функция reduce, которая возвращает пользователя, если он встречается только в первом списке
def REDUCE(key:User, setList:list):
  if len(setList) == 1 and setList[0] == 0:  # Если пользователь встречается только в первом списке
    yield (key, key)  # Возвращаем пару (ключ, ключ)

# Применяем функции RECORDREADER, MAP и REDUCE к нашим данным
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)  # Преобразуем результат в список
output  # Выводим результат

[(User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female')),
 (User(id=2, age=25, social_contacts=240, gender='female'),
  User(id=2, age=25, social_contacts=240, gender='female')),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female'))]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [39]:
# Определяем класс GenderSalary с полями gender и salary
class GenderSalary(NamedTuple):
  gender: str
  salary: int

# Создаем список пользователей
users_list = [
    UserProjection(id=0, gender='male'),
    UserProjection(id=1, gender='female'),
    UserProjection(id=2, gender='female'),
    UserProjection(id=3, gender='female')
]

# Создаем список зарплат
salary_list = [
    GenderSalary(gender='male', salary=200),
    GenderSalary(gender='female', salary=250)
]

# Объединяем два списка в один
data_list = [users_list, salary_list]

def RECORDREADER():  # Функция для чтения данных
  for idx, list_ in enumerate(data_list):  # Проходим по каждому списку в общем списке
    for obj in list_:  # Проходим по каждому объекту в списке
      yield (idx, obj)  # Возвращаем пару (индекс списка, объект)

# Функция map, которая возвращает данные в зависимости от их типа
def MAP(setId:int, obj:NamedTuple):
  if setId == 0:  # Если объект - пользователь
    yield (obj.gender, (0, obj.id))  # Возвращаем пару (пол пользователя, (0, id пользователя))
  else:  # Если объект - зарплата
    yield (obj.gender, (1, obj.salary))  # Возвращаем пару (пол, (1, зарплата))

# Функция reduce, которая возвращает данные, если они соответствуют определенному условию
def REDUCE(key:str, data:list):
  data_R = [a for (setId, a) in data if setId == 0]  # Список id пользователей
  data_S = [c for (setId, c) in data if setId == 1]  # Список зарплат
  for a in data_R:  # Для каждого id пользователя
    for c in data_S:  # Для каждой зарплаты
      yield (a, key, c)  # Возвращаем тройку (id пользователя, пол, зарплата)

# Применяем функции RECORDREADER, MAP и REDUCE к нашим данным
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)  # Преобразуем результат в список
output  # Выводим результат

[(0, 'male', 200), (1, 'female', 250), (2, 'female', 250), (3, 'female', 250)]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примените аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [40]:
class GenderSalaryByGroup(NamedTuple):  # Определяем класс GenderSalaryByGroup с полями groupId, salary и gender
  groupId: int
  salary: int
  gender: str

salary_list1 = [  # Создаем первый список зарплат
    GenderSalary(gender='male', salary=900),
    GenderSalary(gender='female', salary=250),
    GenderSalary(gender='female', salary=150),
    GenderSalary(gender='male', salary=50),
    GenderSalary(gender='male', salary=850)
]

salary_list2 = [  # Создаем второй список зарплат
    GenderSalary(gender='male', salary=200),
    GenderSalary(gender='female', salary=350),
    GenderSalary(gender='female', salary=550),
    GenderSalary(gender='male', salary=650),
    GenderSalary(gender='male', salary=50)
]

data_list = [salary_list1, salary_list2]  # Объединяем два списка зарплат в один

def RECORDREADER():  # Функция для чтения данных
  for idx, list_ in enumerate(data_list):  # Проходим по каждому списку в общем списке
    for gSalary in list_:  # Проходим по каждому объекту в списке
      yield (idx, GenderSalaryByGroup(idx, gSalary.salary, gSalary.gender))  # Возвращаем пару (индекс списка, объект)

def MAP(groupId:int, gSalary:GenderSalaryByGroup):  # Функция map, которая возвращает данные в зависимости от их типа
  yield (groupId, gSalary.salary)  # Возвращаем пару (id группы, зарплата)

def REDUCE(setId:int, data:list):  # Функция reduce, которая возвращает данные, если они соответствуют определенному условию
  yield (setId, sum(data))  # Возвращаем пару (id группы, сумма зарплат)

output = MapReduce(RECORDREADER, MAP, REDUCE)  # Применяем функции RECORDREADER, MAP и REDUCE к нашим данным
output = list(output)  # Преобразуем результат в список
output  # Выводим результат

[(0, 2200), (1, 1800)]

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [41]:
import numpy as np  # Импортируем библиотеку numpy

mat = np.ones((4, 4))  # Создаем матрицу размером 4x4, заполненную единицами
vec = np.array([2, 4, 1, 1])  # Создаем вектор
maps = 3  # Количество карт
reducers = 1  # Количество редьюсеров

def INPUTFORMAT():  # Функция для форматирования входных данных
  global maps  # Объявляем переменную maps глобальной

  def RECORDREADER(split):  # Функция для чтения данных
    for i in range(split.shape[0]):  # Проходим по каждой строке в split
      for j in range(split.shape[1]):  # Проходим по каждому столбцу в split
        yield ((i, j), (split[i, j], vec[j], split.shape[1]))  # Возвращаем пару ((i, j), (значение в split, значение в vec, количество столбцов в split))
  split_size = int(np.ceil(len(mat) / maps))  # Размер каждого split
  for i in range(0, len(mat), split_size):  # Проходим по каждому split в mat
    yield RECORDREADER(mat[i: i + split_size])  # Возвращаем результат функции RECORDREADER для каждого split

def MAP(coordinates:(int, int), values:(int, int, int)):  # Функция map
  i, j = coordinates  # Координаты
  mat_val, vec_val, cols = values  # Значения
  yield ((i, cols), mat_val * vec_val)  # Возвращаем пару ((i, cols), произведение mat_val и vec_val)

def REDUCE(keys:(int, int), products:Iterator[NamedTuple]):  # Функция reduce
  i, cols = keys  # Ключи
  yield (i, sum(products) / (len(products) / cols))  # Возвращаем пару (i, среднее значение products)

# Применяем функции INPUTFORMAT, MAP и REDUCE к нашим данным
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]  # Преобразуем результат в список
partitioned_output  # Выводим результат

16 key-value pairs were sent over a network.


[(0, [(0, 8.0), (1, 8.0)])]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [42]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [43]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I, J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J, K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j, k), big_mat[j, k])

# coordinates - индекс элемента (строка, столбец), value - его значение
def MAP(coordinates:(int, int), value:int):
  j, k = coordinates
  # бегаем по строкам в маленькой матрице
  # индекс строки в новой матрице - индекс строки в маленькой матрице, индекс столбца новой матрицы - индекс столбца в большой матрице
  for row in range(small_mat.shape[0]):
    yield ((row, k), value * small_mat[row][j])

def REDUCE(key, values):
  yield (key, sum(values))

Проверьте своё решение

In [44]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [45]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [46]:
def RECORDREADER():  # Функция для чтения данных
  for k in range(big_mat.shape[1]):  # Проходим по каждому столбцу в big_mat
    for i in range(small_mat.shape[0]):  # Проходим по каждой строке в small_mat
      for j in range(small_mat.shape[1]):  # Проходим по каждому столбцу в small_mat
        yield ((i, k), (0, small_mat[i, j]))  # Возвращаем пару ((i, k), (0, значение в small_mat))
  for i in range(small_mat.shape[0]):  # Проходим по каждой строке в small_mat
    for j in range(big_mat.shape[0]):  # Проходим по каждой строке в big_mat
      for k in range(big_mat.shape[1]):  # Проходим по каждому столбцу в big_mat
        yield ((i, k), (1, big_mat[j, k]))  # Возвращаем пару ((i, k), (1, значение в big_mat))

def MAP(coordinates:(int, int), values:(int, int)):  # Функция map
  yield (coordinates, values)  # Возвращаем пару (координаты, значения)

def REDUCE(coordinates:(int, int), values:list):  # Функция reduce
  small_mat_vals = [val for mat_idx, val in values if mat_idx == 0]  # Список значений из small_mat
  big_mat_vals = [val for mat_idx, val in values if mat_idx == 1]  # Список значений из big_mat
  yield (coordinates, np.dot(small_mat_vals, big_mat_vals))  # Возвращаем пару (координаты, скалярное произведение списков значений)

output = MapReduce(RECORDREADER, MAP, REDUCE)  # Применяем функции RECORDREADER, MAP и REDUCE к нашим данным
np.allclose(reference_solution, asmatrix(output)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [59]:
maps = 2  # Количество map
reducers = 1  # Количество reducer

def INPUTFORMAT():  # Функция для форматирования входных данных
  global maps  # Объявляем переменную maps глобальной
  small_rows = small_mat.shape[0]  # Количество строк в small_mat
  big_rows = big_mat.shape[0]  # Количество строк в big_mat

  def RECORDREADER_SMALL(split, idx):  # Функция для чтения данных из small_mat
    for k in range(big_mat.shape[1]):  # Проходим по каждому столбцу в big_mat
      for i in range(split.shape[0]):  # Проходим по каждой строке в split
        for j in range(split.shape[1]):  # Проходим по каждому столбцу в split
          yield ((i + idx, k), (0, split[i, j]))  # Возвращаем пару ((i + idx, k), (0, значение в split))
  split_size = int(np.ceil(small_rows / maps))  # Размер каждого split
  for i in range(0, small_rows, split_size):  # Проходим по каждому split в small_mat
    yield RECORDREADER_SMALL(small_mat[i : i + split_size], i)  # Возвращаем результат функции RECORDREADER_SMALL для каждого split

  def RECORDREADER_BIG(split):  # Функция для чтения данных из big_mat
    for i in range(small_rows):  # Проходим по каждой строке в small_mat
      for j in range(split.shape[0]):  # Проходим по каждой строке в split
        for k in range(split.shape[1]):  # Проходим по каждому столбцу в split
          yield ((i, k), (1, split[j, k]))  # Возвращаем пару ((i, k), (1, значение в split))

  split_size = int(np.ceil(big_rows / maps))  # Размер каждого split
  for i in range(0, big_rows, split_size):  # Проходим по каждому split в big_mat
    yield RECORDREADER_BIG(big_mat[i : i + split_size])  # Возвращаем результат функции RECORDREADER_BIG для каждого split

def MAP(coordinates:(int, int), values:(int, int)):  # Функция map
  yield (coordinates, values)  # Возвращаем пару (координаты, значения)

def REDUCE(coordinates:(int, int), values:list):  # Функция reduce
  small_mat_vals = [val for mat_idx, val in values if mat_idx == 0]  # Список значений из small_mat
  big_mat_vals = [val for mat_idx, val in values if mat_idx == 1]  # Список значений из big_mat
  yield (coordinates, np.dot(small_mat_vals, big_mat_vals))  # Возвращаем пару (координаты, скалярное произведение списков значений)

# Применяем функции INPUTFORMAT, MAP и REDUCE к нашим данным
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]  # Преобразуем результат в список

def asmatrix_distributed(result):  # Функция для преобразования результата в матрицу
  result_list = []  # Список для хранения результатов
  for _, values in result:  # Проходим по каждому значению в result
    for val in values:  # Проходим по каждому значению в values
      result_list.append(val)  # Добавляем значение в result_list
  count = {}  # Словарь для хранения результатов
  for k, v in result_list:  # Проходим по каждому элементу в result_list
    if k in count:  # Если ключ уже есть в словаре
      count[k] += v  # Увеличиваем значение на v
    else:  # Если ключа нет в словаре
      count[k] = v  # Добавляем ключ и значение в словарь
  return asmatrix((k, v) for k, v in count.items())  # Возвращаем матрицу, созданную из элементов словаря

np.allclose(reference_solution, asmatrix_distributed(partitioned_output)) # should return true

480 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

## Решение для блочного разбиения матрицы

In [58]:
maps = 2  # Количество map
reducers = 1  # Количество reducer

def INPUTFORMAT():  # Функция для форматирования входных данных
  global maps  # Объявляем переменную maps глобальной
  small_rows = small_mat.shape[0]  # Количество строк в small_mat
  small_cols = small_mat.shape[1]  # Количество столбцов в small_mat
  big_rows = big_mat.shape[0]  # Количество строк в big_mat
  big_cols = big_mat.shape[1]  # Количество столбцов в big_mat

  def RECORDREADER_SMALL1(split, idx):  # Функция для чтения данных из small_mat
    for k in range(big_cols):  # Проходим по каждому столбцу в big_mat
      for i in range(split.shape[0]):  # Проходим по каждой строке в split
        for j in range(split.shape[1]):  # Проходим по каждому столбцу в split
          yield ((i + idx, k), (0, split[i, j]))  # Возвращаем пару ((i + idx, k), (0, значение в split))

  def RECORDREADER_SMALL2(split, idx):  # Функция для чтения данных из small_mat
    for k in range(big_cols):  # Проходим по каждому столбцу в big_mat
      for i in range(split.shape[0]):  # Проходим по каждой строке в split
        for j in range(split.shape[1]):  # Проходим по каждому столбцу в split
          yield ((i + idx, k), (0, split[i, j]))  # Возвращаем пару ((i + idx, k), (0, значение в split))

  def RECORDREADER_BIG1(split, idx):  # Функция для чтения данных из big_mat
    for i in range(small_rows):  # Проходим по каждой строке в small_mat
      for j in range(split.shape[0]):  # Проходим по каждой строке в split
        for k in range(split.shape[1]):  # Проходим по каждому столбцу в split
          yield ((i, k + idx), (1, split[j, k]))  # Возвращаем пару ((i, k + idx), (1, значение в split))

  def RECORDREADER_BIG2(split, idx):  # Функция для чтения данных из big_mat
    for i in range(small_rows):  # Проходим по каждой строке в small_mat
      for j in range(split.shape[0]):  # Проходим по каждой строке в split
        for k in range(split.shape[1]):  # Проходим по каждому столбцу в split
           yield ((i, k + idx), (1, split[j, k]))  # Возвращаем пару ((i, k + idx), (1, значение в split))

  small_readers = {  # Словарь функций чтения данных из small_mat
      0: RECORDREADER_SMALL1,
      1: RECORDREADER_SMALL2
  }

  big_readers = {  # Словарь функций чтения данных из big_mat
       0: RECORDREADER_BIG1,
       1: RECORDREADER_BIG2
  }

  split_size_rows = int(np.ceil(small_rows / maps))  # Размер каждого split по строкам
  split_size_cols = int(np.ceil(small_cols / maps))  # Размер каждого split по столбцам
  counter = -1  # Счетчик
  for i in range(0, small_rows, split_size_rows):  # Проходим по каждому split в small_mat по строкам
    for j in range(0, small_cols, split_size_cols):  # Проходим по каждому split в small_mat по столбцам
      counter += 1  # Увеличиваем счетчик
      yield small_readers[counter % len(small_readers)](small_mat[i : i + split_size_rows, j : j + split_size_cols], i)

  split_size_rows = int(np.ceil(big_rows / maps))  # Размер каждого split по строкам
  split_size_cols = int(np.ceil(big_cols / maps))  # Размер каждого split по столбцам
  counter = -1  # Счетчик
  for i in range(0, big_rows, split_size_rows):  # Проходим по каждому split в big_mat по строкам
    for j in range(0, big_cols, split_size_cols):  # Проходим по каждому split в big_mat по столбцам
      counter += 1  # Увеличиваем счетчик
      yield big_readers[counter % len(big_readers)](big_mat[i : i + split_size_rows, j : j + split_size_cols], j)

def MAP(coordinates:(int, int), values:(int, int)):  # Функция map
  yield (coordinates, values)  # Возвращаем пару (координаты, значения)

def REDUCE(coordinates:(int, int), values:list):  # Функция reduce
  small_mat_vals = [val for mat_idx, val in values if mat_idx == 0]  # Список значений из small_mat
  big_mat_vals = [val for mat_idx, val in values if mat_idx == 1]  # Список значений из big_mat
  yield (coordinates, np.dot(small_mat_vals, big_mat_vals))  # Возвращаем пару (координаты, скалярное произведение списков значений)

# Применяем функции INPUTFORMAT, MAP и REDUCE к нашим данным
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]  # Преобразуем результат в список

np.allclose(reference_solution, asmatrix_distributed(partitioned_output)) # should return true

480 key-value pairs were sent over a network.


True

## Решение для рандомного распределения индексов


In [53]:
import random  # Импортируем модуль random

def shuffle_idxs(mat):  # Функция для перемешивания индексов матрицы
  idxs_list = [(i, j) for i in range(mat.shape[0]) for j in range(mat.shape[1])]  # Создаем список пар индексов для каждого элемента матрицы
  random.shuffle(idxs_list)  # Перемешиваем список пар индексов
  return idxs_list  # Возвращаем перемешанный список пар индексов

### Рандомное распределение с использованием такого же кода REDUCER'ов и RECORDREADER'ов, как выше

In [57]:
maps = 2  # Количество map
reducers = 1  # Количество reducer

def INPUTFORMAT():  # Функция для форматирования входных данных
  global maps  # Объявляем переменную maps глобальной
  small_rows = small_mat.shape[0]  # Количество строк в small_mat
  small_cols = small_mat.shape[1]  # Количество столбцов в small_mat
  big_rows = big_mat.shape[0]  # Количество строк в big_mat
  big_cols = big_mat.shape[1]  # Количество столбцов в big_mat

  def RECORDREADER_SMALL1(split, idx):  # Функция для чтения данных из small_mat
    for k in range(big_cols):  # Проходим по каждому столбцу в big_mat
      for i in range(split.shape[0]):  # Проходим по каждой строке в split
        for j in range(split.shape[1]):  # Проходим по каждому столбцу в split
          yield ((i + idx, k), (0, split[i, j]))  # Возвращаем пару ((i + idx, k), (0, значение в split))

  def RECORDREADER_SMALL2(split, idx):  # Функция для чтения данных из small_mat
    for k in range(big_cols):  # Проходим по каждому столбцу в big_mat
      for i in range(split.shape[0]):  # Проходим по каждой строке в split
        for j in range(split.shape[1]):  # Проходим по каждому столбцу в split
          yield ((i + idx, k), (0, split[i, j]))  # Возвращаем пару ((i + idx, k), (0, значение в split))

  def RECORDREADER_BIG1(split, idx):  # Функция для чтения данных из big_mat
    for i in range(small_rows):  # Проходим по каждой строке в small_mat
      for j in range(split.shape[0]):  # Проходим по каждой строке в split
        for k in range(split.shape[1]):  # Проходим по каждому столбцу в split
          yield ((i, k + idx), (1, split[j, k]))  # Возвращаем пару ((i, k + idx), (1, значение в split))

  def RECORDREADER_BIG2(split, idx):  # Функция для чтения данных из big_mat
    for i in range(small_rows):  # Проходим по каждой строке в small_mat
      for j in range(split.shape[0]):  # Проходим по каждой строке в split
        for k in range(split.shape[1]):  # Проходим по каждому столбцу в split
           yield ((i, k + idx), (1, split[j, k]))  # Возвращаем пару ((i, k + idx), (1, значение в split))

  small_readers = {  # Словарь функций чтения данных из small_mat
      0: RECORDREADER_SMALL1,
      1: RECORDREADER_SMALL2
  }
  big_readers = {  # Словарь функций чтения данных из big_mat
       0: RECORDREADER_BIG1,
       1: RECORDREADER_BIG2
  }
  counter = -1  # Счетчик
  small_idxs = shuffle_idxs(small_mat)  # Перемешиваем индексы small_mat
  for (i, j) in small_idxs:  # Проходим по каждому индексу в small_idxs
    counter += 1  # Увеличиваем счетчик
    yield small_readers[counter % len(small_readers)](np.array([[small_mat[i, j]]]), i)

  counter = -1  # Счетчик
  big_idxs = shuffle_idxs(big_mat)  # Перемешиваем индексы big_mat
  for (i, j) in big_idxs:  # Проходим по каждому индексу в big_idxs
    counter += 1  # Увеличиваем счетчик
    yield big_readers[counter % len(big_readers)](np.array([[big_mat[i, j]]]), j)

def MAP(coordinates:(int, int), values:(int, int)):  # Функция map
  yield (coordinates, values)  # Возвращаем пару (координаты, значения)

def REDUCE(coordinates:(int, int), values:list):  # Функция reduce
  small_mat_vals = [val for mat_idx, val in values if mat_idx == 0]  # Список значений из small_mat
  big_mat_vals = [val for mat_idx, val in values if mat_idx == 1]  # Список значений из big_mat
  yield (coordinates, np.dot(small_mat_vals, big_mat_vals))  # Возвращаем пару (координаты, скалярное произведение списков значений)

# Применяем функции INPUTFORMAT, MAP и REDUCE к нашим данным
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]  # Преобразуем результат в список

np.allclose(reference_solution, asmatrix_distributed(partitioned_output)) # should return true

480 key-value pairs were sent over a network.


False

Как видно, подход с использованием RECORDREADER-ов для генерации случайного подмножества элементов матрицы окажется работоспобным, если конечная последовательность элементов после их случайного выбора будет соответствовать исходному порядку элементов в матрице.

### Рандомное распределение с использованием доработанного кода REDUCER'ов и RECORDREADER'ов

In [56]:
maps = 2  # Количество map
reducers = 1  # Количество reducer

def INPUTFORMAT():  # Функция для форматирования входных данных
  global maps  # Объявляем переменную maps глобальной
  small_rows = small_mat.shape[0]  # Количество строк в small_mat
  small_cols = small_mat.shape[1]  # Количество столбцов в small_mat
  big_rows = big_mat.shape[0]  # Количество строк в big_mat
  big_cols = big_mat.shape[1]  # Количество столбцов в big_mat

  def RECORDREADER_SMALL1(split, idxs):  # Функция для чтения данных из small_mat
    row, col = idxs  # Индексы строки и столбца
    for k in range(big_cols):  # Проходим по каждому столбцу в big_mat
      for i in range(split.shape[0]):  # Проходим по каждой строке в split
        for j in range(split.shape[1]):  # Проходим по каждому столбцу в split
          yield ((i + row, k), (0, col, split[i, j]))  # Возвращаем пару ((i + row, k), (0, col, значение в split))

  def RECORDREADER_SMALL2(split, idxs):  # Функция для чтения данных из small_mat
    row, col = idxs  # Индексы строки и столбца
    for k in range(big_cols):  # Проходим по каждому столбцу в big_mat
      for i in range(split.shape[0]):  # Проходим по каждой строке в split
        for j in range(split.shape[1]):  # Проходим по каждому столбцу в split
          yield ((i + row, k), (0, col, split[i, j]))  # Возвращаем пару ((i + row, k), (0, col, значение в split))

  def RECORDREADER_BIG1(split, idxs):  # Функция для чтения данных из big_mat
    row, col = idxs  # Индексы строки и столбца
    for i in range(small_rows):  # Проходим по каждой строке в small_mat
      for j in range(split.shape[0]):  # Проходим по каждой строке в split
        for k in range(split.shape[1]):  # Проходим по каждому столбцу в split
          yield ((i, k + col), (1, row, split[j, k]))  # Возвращаем пару ((i, k + col), (1, row, значение в split))

  def RECORDREADER_BIG2(split, idxs):  # Функция для чтения данных из big_mat
    row, col = idxs  # Индексы строки и столбца
    for i in range(small_rows):  # Проходим по каждой строке в small_mat
      for j in range(split.shape[0]):  # Проходим по каждой строке в split
        for k in range(split.shape[1]):  # Проходим по каждому столбцу в split
           yield ((i, k + col), (1, row, split[j, k]))  # Возвращаем пару ((i, k + col), (1, row, значение в split))

  small_readers = {  # Словарь функций чтения данных из small_mat
      0: RECORDREADER_SMALL1,
      1: RECORDREADER_SMALL2
  }
  big_readers = {  # Словарь функций чтения данных из big_mat
       0: RECORDREADER_BIG1,
       1: RECORDREADER_BIG2
  }
  counter = -1  # Счетчик
  small_idxs = shuffle_idxs(small_mat)  # Перемешиваем индексы small_mat
  for (i, j) in small_idxs:  # Проходим по каждому индексу в small_idxs
    counter += 1  # Увеличиваем счетчик
    yield small_readers[counter % len(small_readers)](np.array([[small_mat[i, j]]]), (i, j))

  counter = -1  # Счетчик
  big_idxs = shuffle_idxs(big_mat)  # Перемешиваем индексы big_mat
  for (i, j) in big_idxs:  # Проходим по каждому индексу в big_idxs
    counter += 1  # Увеличиваем счетчик
    yield big_readers[counter % len(big_readers)](np.array([[big_mat[i, j]]]), (i, j))

def MAP(coordinates:(int, int), values:(int, int, int)):  # Функция map
  yield (coordinates, values)  # Возвращаем пару (координаты, значения)

def REDUCE(coordinates:(int, int), values:list):  # Функция reduce
  sorted_values = sorted(values, key=lambda x: x[1])  # Сортируем значения по второму элементу каждого значения
  small_mat_vals = [val for mat_idx, col, val in sorted_values if mat_idx == 0]  # Список значений из small_mat
  big_mat_vals = [val for mat_idx, row, val in sorted_values if mat_idx == 1]  # Список значений из big_mat
  yield (coordinates, np.dot(small_mat_vals, big_mat_vals))  # Возвращаем пару (координаты, скалярное произведение списков значений)

# Применяем функции INPUTFORMAT, MAP и REDUCE к нашим данным
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]  # Преобразуем результат в список

np.allclose(reference_solution, asmatrix_distributed(partitioned_output)) # should return true

480 key-value pairs were sent over a network.


True

Можно заметить, что метод, при котором RECORDREADER-ы создают случайное подмножество элементов из матрицы, окажется работоспособным, если вместе с значением и индексом в RECORDREADER также будет включена информация о расположении элемента в пределах строки или матрицы.